In [1]:
import automol

rsmi, psmi = ("CCCO[O]", "[CH2]CCOO")
rsmis = automol.smiles.split(rsmi)
psmis = automol.smiles.split(psmi)

# Get the starting TS structure
rxn, *_ = automol.reac.from_smiles(rsmis, psmis, stereo=False, struc_typ="geom")
ts_gra = automol.reac.ts_graph(rxn)
ts_geo = automol.reac.ts_structure(rxn)

# Determine the breaking and forming bonds
brk_bkey, = automol.graph.ts.breaking_bond_keys(ts_gra)
frm_bkey, = automol.graph.ts.breaking_bond_keys(ts_gra)

automol.graph.display(ts_gra, label=True, exp=True)
# automol.geom.display(ts_geo, gra=ts_gra)

In [2]:
from tblite.ase import TBLite
from ase import constraints
from ase.optimize import climbfixinternals

atms_obj = automol.geom.ase_atoms(ts_geo)
atms_obj.calc = TBLite(method="GFN1-xTB")

# Define reaction coordinate via linear combination of bond lengths
reaction_coord = [[*brk_bkey, 1.0], [*frm_bkey, 1.0]]
# Use current value `FixInternals.get_combo(atoms, reaction_coord)`
# as initial value

bondcombo = [None, reaction_coord]  # 'None' will converts to current value
atms_obj.set_constraint([constraints.FixInternals(bondcombos=[bondcombo])])

# AVC: INSTEAD, START WITH A SIMPLE CONSTRAINED OPTIMIZATION

# Optimizer for transition state search along reaction coordinate
opt = climbfixinternals.BFGSClimbFixInternals(atms_obj, climb_coordinate=reaction_coord)
opt.run(fmax=0.05)  # Converge to a saddle point

# # Validate transition state by one imaginary vibrational mode
# vib = Vibrations(atoms, indices=[4])
# vib.run()
# assert ((np.imag(vib.get_energies()) > 0) == [True, False, False]).all()

geo_out = automol.geom.from_ase_atoms(atms_obj)
automol.geom.display(geo_out)

------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -18.99057833730  -1.9167150E+01   5.3793479E-01
      2     -19.21567441147  -2.2509607E-01   3.3989348E-01
      3     -19.20704254664   8.6318648E-03   1.6074957E-01
      4     -19.22777691335  -2.0734367E-02   5.6557343E-02
      5     -19.23666016733  -8.8832540E-03   2.6427568E-02
      6     -19.23930951642  -2.6493491E-03   4.2408560E-03
      7     -19.23929384216   1.5674262E-05   2.6364911E-03
      8     -19.23929202588   1.8162756E-06   2.8004935E-03
      9     -19.23932836414  -3.6338255E-05   3.8978647E-04
     10     -19.23932889673  -5.3258706E-07   2.9796322E-04
     11     -19.23932929923  -4.0250321E-07   1.0188038E-04
     12     -19.23932935164  -5.2415654E-08   1.4591912E-05
------------------------------------------------------------

 total:                             

Exception ignored from cffi callback <function logger_callback at 0x7f1c41526980>:
Traceback (most recent call last):
  File "/home/avcopan/code/amech-dev/.pixi/envs/default/lib/python3.12/site-packages/tblite/library.py", line 78, in logger_callback
    @ffi.def_extern()
    
KeyboardInterrupt: 


      3     -19.27853752056  -5.7706990E-05   8.6364702E-04
      5     -19.27853786371  -1.9319983E-07   1.5945540E-04
      6     -19.27853788294  -1.9234978E-08   1.2514700E-05
------------------------------------------------------------

 total:                                   0.866 sec
------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -19.27934174795  -1.9484122E+01   3.2915329E-03
      2     -19.27935576383  -1.4015886E-05   1.6624991E-03
      3     -19.27936181980  -6.0559694E-06   7.5628689E-04
      4     -19.27936194469  -1.2488323E-07   2.1860819E-04
      5     -19.27936201406  -6.9370483E-08   4.3791931E-05
      6     -19.27936201538  -1.3266970E-09   1.1861681E-05
------------------------------------------------------------

 total:                                   0.662 sec
--------------------------------------------------